# Background
This notebook goes through the off position reduction in class for the orion B map

In [1]:
from IPython.display import HTML

HTML('''
// http://lokeshdhakar.com/projects/lightbox2
<link href="./notebook_support/lightbox2-master/dist/css/lightbox.css" rel="stylesheet">

<script src="./notebook_support/lightbox2-master/dist/js/lightbox.js"></script>

<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

# import commands

In [121]:
## import
from astropy.io import fits
import numpy as np
import glob
import datetime
import pandas as pd
import logging
import os,re
import time
import subprocess
from matplotlib.lines import lineStyles
import jinja2
import requests

# notebook
from IPython.display import Image, display
from IPython.display import HTML
# astropy
from astropy.io import fits
from astropy import constants as const
from astropy import units as u


def PrintMessage(message):
    html='''
    <table style="width:100%">
        <tr>
            <td style='text-align:center;vertical-align:middle'>{0}</td>
        </tr>
    </table>
    '''
    display(HTML(html.format(message)))


def Change1dListTo2dList(array, column=2):
    test = []
    test_row = []
    for item in array:
        test_row.append(item)
        if len(test_row)> column-1:
            test.append(test_row)
            test_row = []
    if len(test_row)>0:
        test.append(test_row)
    return test

def DisplayImagesInColumns(images,image_columns=2, gallery_label=""):
    popup_css='''    
    
    <style>
    .thumb img { 
        border:1px solid #000;
        margin:3px;    
        float:left;
    }
    .thumb span { 
        position:fixed;
        visibility:hidden;
    }
    .thumb:hover, .thumb:hover span { 
        visibility:visible;
        z-index:1;
    }

    </style>

    '''
    
    html='''    
    <a href={0} target="_blank">
    <img style='width: {1:2.0f}%; margin: 0px; float: left; border: 0px solid black;' src='{0}?{2}' />
    </a>
    '''
    html='''    
    <a href={0}  target="_blank">
    <img style='width: {1:2.0f}%; margin: 0px; float: left; border: 0px solid black;' src='{0}?{2}' />
    </a>
    '''
    
    #html='''    
    #<a href={0}  data-lightbox={3} data-title="{0}">
    #<img style='width: {1:2.0f}%; margin: 0px; float: left; border: 0px solid black;' src='{0}?{2}' />
    #</a>
    #'''
    
    list2d = Change1dListTo2dList(images, column=image_columns)
    width_percent = (1/float(image_columns))*100.0
    gallery_label = gallery_label.replace(" ","")
    for row in list2d:
        imagesList=popup_css.join( [html.format(image,width_percent-2,time.time(),gallery_label) 
                     for image in row ])
        display(HTML(imagesList))
    return imagesList
   

# Helper Class to run class_cmd_strings and export the images that were created to the notebook
class RunClassScript():
    def __init__(self, cmd, no_output=False, display_images=True, test=False):
        self.cmd = cmd
        self.cmd_file = "/tmp/cmd.class"
        self.cmd_log = "/tmp/cmd.log"
        self.images = [string.split()[1].replace("\"","").replace("//","/") for string in self.cmd.split("\n") if string[0:2] == "ha"]
        if self.cmd.split("\n")[-1].strip() != "exit 0":
            self.cmd = "{}\nexit 0\n".format(self.cmd)
        subprocess.call(["rm", "-rf", "/tmp/cmd.class"])
        fileout = open(self.cmd_file, "w")
        fileout.write(self.cmd)
        fileout.close()
        #output = subprocess.check_output("class -nw @{0}".format(self.cmd_file), shell=True)
        if test:
            print(self.cmd)
        try:
            output = subprocess.check_call("class -nw @{0} > {1}".format(self.cmd_file,self.cmd_log), shell=True)
        except subprocess.CalledProcessError as e:
            print("======================================")
            print("=========== ERROR in class cmd =======")
            print("======================================")
            print("\n")
            print("======================================")
            print("============= INPUT FILE =============")
            print("======================================")
            with open(self.cmd_file, 'r') as fin:
                for line in fin.readlines():
                    if line[0]!=" ":
                        print(line.strip("\n"))
            print("======================================")
            print("========= OUTPUT LOGFILE =============")
            print("======================================")                        
            with open(self.cmd_log, 'r') as fin:
                for line in fin.readlines():
                    if (len(set(line))>1):
                        print(line.strip("\n"))
            raise subprocess.CalledProcessError(e.returncode,e.cmd, e.output)
        if output!=0:
            print(output)
            self.output=output
        #
        if display_images:
            for image in self.images:
                if os.path.exists(image):
                    display(image, Image(image),width=300,height=300)
                    
def GenerateGIFAnimation(images,gif_filename):
    convert_cmd = "convert -delay 50"\
                  " -loop 0 "\
                  "{0} "\
                  "{1}".format(images,gif_filename)
    os.system(convert_cmd)
    

    from IPython.display import HTML
    

def DownloadFile(url,local_filename):
    #local_filename = url.split('/')[-1]
    r = requests.get(url)
    f = open(local_filename, 'wb')
    for chunk in r.iter_content(chunk_size=512 * 1024): 
        if chunk: # filter out keep-alive new chunks
            f.write(chunk)
    f.close()
    return 

# Download test data from archive



In [117]:
data_folder = "./tmp/data/"


if not os.path.exists(data_folder):
    print("making folder {0}".format(data_folder))
    #os.makedirs(data_folder)
    os.makedirs(data_folder,exist_ok=True)
    
urls = []
#urls.append("https://irsa.ipac.caltech.edu/data/SOFIA/GREAT/OC4K/20161110_F348/proc/p8486/data/2016-11-10_GR_F348_04_0066_0010505_1900536.9.great.tar")
#urls.append("https://irsa.ipac.caltech.edu/data/SOFIA/GREAT/OC5A/20170208_F371/proc/p4006/data/2017-02-08_GR_F371_04_0066_0010505_1900536.9.great.tar")
urls.append("https://irsa.ipac.caltech.edu/data/SOFIA/GREAT/OC4K/20161116_F350/proc/p8488/data/2016-11-16_GR_F350_04_0066_0010506_1900536.9.great.tar")

for url in urls:
    filename = "{0}/{1}".format(data_folder,url.split('/')[-1])
    if not os.path.exists(filename):
        print("downloading {0} to {1}".format(url, filename))
        download_file(url, filename)
    else:
        print("{1} already downloaded".format(url, filename))
# untar
already_downloaded_great = glob.glob("tmp/data/**/*.great")
#
for url in urls:
    # check for great file
    tar_filename = "{0}/{1}".format(data_folder,url.split('/')[-1])
    filename_great,ext = os.path.splitext(tar_filename)
    full_filename_great,ext = os.path.splitext(os.path.basename(filename_great))
    #
    found = [filename for filename in  already_downloaded_great if full_filename_great in filename]
    if len(found)>0:
        print(" {0} already extracted".format(full_filename_great))
    else:
        print(" {0} extracting".format(full_filename_great))
        cmd="tar -xf {0} --wildcards --no-anchored '*Tmb.great' --one-top-level={1}".format(tar_filename,data_folder)
        print(cmd)
        output = subprocess.check_call(cmd, shell=True)

downloading https://irsa.ipac.caltech.edu/data/SOFIA/GREAT/OC4K/20161116_F350/proc/p8488/data/2016-11-16_GR_F350_04_0066_0010506_1900536.9.great.tar to ./tmp/data//2016-11-16_GR_F350_04_0066_0010506_1900536.9.great.tar
 2016-11-16_GR_F350_04_0066_0010506_1900536.9 extracting
tar -xf ./tmp/data//2016-11-16_GR_F350_04_0066_0010506_1900536.9.great.tar --wildcards --no-anchored '*Tmb.great' --one-top-level=./tmp/data/


# generate average pixel plot

In [113]:
class_script='''
@kosma-init
file in "{filename}"
find
set unit v
set mode x {min_x} {max_x}
set mode y {min_y} {max_y}
set line {line}
set source {source}
set window {window}

find 

go where
draw t 15 20 {plot_filename} 6
ha "{plot_filename}_go_where.png" /dev png /overwrite

plot_group /group_by telescope
draw t 15 20 {plot_filename} 6
ha "{plot_filename}" /dev png /overwrite
'''

In [114]:
plots_folder = "tmp/plots"
os.makedirs(plots_folder,exist_ok=True)
#
class_cmd = {}
class_cmd['min_x'] = -40
class_cmd['max_x'] = 40
class_cmd['min_y'] = -30
class_cmd['max_y'] = 130
class_cmd['smooth'] = 10
class_cmd['line'] = "CII_L"
class_cmd['source'] = "OMC_CENTER"
class_cmd['window'] = "-20 20"

already_downloaded_great = glob.glob("tmp/data/**/*.great")
images = []
for class_filename in already_downloaded_great:
    print(class_filename)
    class_cmd["filename"] = os.path.abspath(class_filename)    
    class_cmd["plot_filename"] = "{0}/plot_group_{1}.png".format(plots_folder,os.path.basename(class_filename))
    print(class_cmd["plot_filename"])
    cmd=class_script.format(**class_cmd)
    output = RunClassScript(cmd, no_output=False,display_images=False)
    images.extend(output.images)
message = "Test message"
PrintMessage(message)
DisplayImagesInColumns(images,2,'test_gallery')

tmp/data/2017-02-08_GR_F371_04_0066_0010505_1900536.9/2017-02-08_GR_F371_04_0066_0010505_1900536.9_Tmb.great
tmp/plots/plot_group_2017-02-08_GR_F371_04_0066_0010505_1900536.9_Tmb.great.png
tmp/data/2016-11-10_GR_F348_04_0066_0010505_1900536.9/2016-11-10_GR_F348_04_0066_0010505_1900536.9_Tmb.great
tmp/plots/plot_group_2016-11-10_GR_F348_04_0066_0010505_1900536.9_Tmb.great.png


Test message


'    \n    <a href=tmp/plots/plot_group_2016-11-10_GR_F348_04_0066_0010505_1900536.9_Tmb.great.png_go_where.png  target="_blank">\n    <img style=\'width: 48%; margin: 0px; float: left; border: 0px solid black;\' src=\'tmp/plots/plot_group_2016-11-10_GR_F348_04_0066_0010505_1900536.9_Tmb.great.png_go_where.png?1643635253.762815\' />\n    </a>\n        \n    \n    <style>\n    .thumb img { \n        border:1px solid #000;\n        margin:3px;    \n        float:left;\n    }\n    .thumb span { \n        position:fixed;\n        visibility:hidden;\n    }\n    .thumb:hover, .thumb:hover span { \n        visibility:visible;\n        z-index:1;\n    }\n\n    </style>\n\n        \n    <a href=tmp/plots/plot_group_2016-11-10_GR_F348_04_0066_0010505_1900536.9_Tmb.great.png  target="_blank">\n    <img style=\'width: 48%; margin: 0px; float: left; border: 0px solid black;\' src=\'tmp/plots/plot_group_2016-11-10_GR_F348_04_0066_0010505_1900536.9_Tmb.great.png?1643635253.7628303\' />\n    </a>\n   

In [116]:
print(output.cmd_log)

/tmp/cmd.log


# Generate overview plots
Here is an example of simple script to plot a single class spectra and display this in the notebook